# Camada refined
Transformações para obtenção de zona de características para treinamento do modelo

* Padronização em lower case
* Remover palavras comuns e conectores
* Remover espaço em branco e pontuações
* Atribuição de id hash para garantir a rastreabilidade dos textos

### Configs

In [ ]:
pip install gensim

In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_sm

In [1]:
import pandas as pd
from gensim.utils import simple_preprocess

### Leitura da camada Trusted

In [2]:
df_claims = pd.read_parquet('../dataset/trusted/telecom_pandemic_claims.parquet.snappy')
df_claims

,company_name,date,uf,city,customer_claim,company_response
42907,Vivo - Telefônica,2020-12-23,MG,Belo Horizonte,O TELEFONE ACIMA CITADO ESTAVA LIGADO À TIM. F...,"Curitiba, 13 de Dezembro de 2020. Protoc..."
43118,Tim,2020-12-30,MG,Belo Horizonte,"Bom dia, Recebi ontem, 29/12/2020, ligação do ...","Prezado Guilherme, boa tarde! Em atenção à su..."
43122,Tim,2020-12-31,MG,Governador Valadares,Usei os canais digitais para migrar meu plano ...,Prezado cliente Sua demanda foi concluída no ...
43123,Tim,2020-12-30,MG,Juiz de Fora,O plano foi cancelado no dia 26/11/2020 e poss...,"""Prezado Jeferson, Em atenção à sua demanda pe..."
43124,Tim,2020-12-30,MG,Lagoa Santa,Trabalho em uma região de Lagoa Santa/MG que o...,Prezado(a) Vinicius ! Em virtude do prazo esti...
...,...,...,...,...,...,...
174220,Tim,2020-02-01,MG,Belo Horizonte,Recebi a conta TIM com vencimento em 10/01/202...,Weber Carneiro Gomes de Lima. Boa tarde. Em a...
174232,Oi Celular,2020-02-01,MG,Além Paraíba,"BOM DIA FIZ O PLANO POS PAGO 59,90 E A PRIMEIR...","Conforme contato realizado no dia 03/01/2020, ..."
174255,Oi Fixo,2020-03-01,MG,Ribeirão das Neves,Estou procurando a empresa para renegociar min...,"Boa dia! Prezado , conforme contato realizado ..."
174716,Claro Celular,2020-02-01,MG,Belo Horizonte,Deixei de ser cliente Claro em novembro de 201...,"Sr.(a) ROMULO DUQUE ESTRADA CARVALHO, Em atenç..."


In [3]:
df_claims.count()

company_name        28294
date                28294
uf                  28294
city                28294
customer_claim      28294
company_response    28288
dtype: int64

In [4]:
df_claims.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28294 entries, 42907 to 175101
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   company_name      28294 non-null  object        
 1   date              28294 non-null  datetime64[ns]
 2   uf                28294 non-null  object        
 3   city              28294 non-null  object        
 4   customer_claim    28294 non-null  object        
 5   company_response  28288 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 1.5+ MB


### Criação Id Hash
Criação de ID hash para garantir rastreabilidade dos registros

In [5]:
import hashlib
def create_hash(row):
    # Concatena todos os valores das colunas e gera o hash sha1
    concatenated_string = ''.join(row.astype(str))
    return hashlib.sha1(concatenated_string.encode('utf-8')).hexdigest()

# Adiciona coluna 'id'
df_claims['id'] = df_claims.apply(create_hash, axis=1)
df_claims

,company_name,date,uf,city,customer_claim,company_response,id
42907,Vivo - Telefônica,2020-12-23,MG,Belo Horizonte,O TELEFONE ACIMA CITADO ESTAVA LIGADO À TIM. F...,"Curitiba, 13 de Dezembro de 2020. Protoc...",9052980eaeff105ac8bb1ba073230ea53ede5ed6
43118,Tim,2020-12-30,MG,Belo Horizonte,"Bom dia, Recebi ontem, 29/12/2020, ligação do ...","Prezado Guilherme, boa tarde! Em atenção à su...",15c8f5eb376228c5aa983cd4d9b33ff42b6cc24a
43122,Tim,2020-12-31,MG,Governador Valadares,Usei os canais digitais para migrar meu plano ...,Prezado cliente Sua demanda foi concluída no ...,8986fa44aa6ff79d316b60ce386f93f67814c2f6
43123,Tim,2020-12-30,MG,Juiz de Fora,O plano foi cancelado no dia 26/11/2020 e poss...,"""Prezado Jeferson, Em atenção à sua demanda pe...",186fab829093dcd81b85f050cddd5f79cd8d958d
43124,Tim,2020-12-30,MG,Lagoa Santa,Trabalho em uma região de Lagoa Santa/MG que o...,Prezado(a) Vinicius ! Em virtude do prazo esti...,690d2f4395b261feeda5b7bded494219c1531b43
...,...,...,...,...,...,...,...
174220,Tim,2020-02-01,MG,Belo Horizonte,Recebi a conta TIM com vencimento em 10/01/202...,Weber Carneiro Gomes de Lima. Boa tarde. Em a...,7b80c7c8ea98acade5233a1865aab6349d019679
174232,Oi Celular,2020-02-01,MG,Além Paraíba,"BOM DIA FIZ O PLANO POS PAGO 59,90 E A PRIMEIR...","Conforme contato realizado no dia 03/01/2020, ...",740e49a6913c9d284b88ed04459d2224058aec83
174255,Oi Fixo,2020-03-01,MG,Ribeirão das Neves,Estou procurando a empresa para renegociar min...,"Boa dia! Prezado , conforme contato realizado ...",bbe695f45625cb1da558c41d13fc6f3c70e0c897
174716,Claro Celular,2020-02-01,MG,Belo Horizonte,Deixei de ser cliente Claro em novembro de 201...,"Sr.(a) ROMULO DUQUE ESTRADA CARVALHO, Em atenç...",d7dbb0babca4eadcc13aade89ec8b38fddebf981


## Tranformações
Utilizando o pacote *simple_preprocess*, responsável por gerar uma lista de tokens

Tranformações aplicadas:
*   Padronização em lower case
*   Desconsideração de palavras com 1 caracter e mais de 15 caracteres
*   Desconsideração de espaços em branco, números e caracteres especiais


Doc: https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [6]:
df_claims['tokens_customer_claim'] = df_claims['customer_claim'].apply(lambda x: simple_preprocess(x, deacc=True))
df_claims

,company_name,date,uf,city,customer_claim,company_response,id,tokens_customer_claim
42907,Vivo - Telefônica,2020-12-23,MG,Belo Horizonte,O TELEFONE ACIMA CITADO ESTAVA LIGADO À TIM. F...,"Curitiba, 13 de Dezembro de 2020. Protoc...",9052980eaeff105ac8bb1ba073230ea53ede5ed6,"[telefone, acima, citado, estava, ligado, tim,..."
43118,Tim,2020-12-30,MG,Belo Horizonte,"Bom dia, Recebi ontem, 29/12/2020, ligação do ...","Prezado Guilherme, boa tarde! Em atenção à su...",15c8f5eb376228c5aa983cd4d9b33ff42b6cc24a,"[bom, dia, recebi, ontem, ligacao, do, numero,..."
43122,Tim,2020-12-31,MG,Governador Valadares,Usei os canais digitais para migrar meu plano ...,Prezado cliente Sua demanda foi concluída no ...,8986fa44aa6ff79d316b60ce386f93f67814c2f6,"[usei, os, canais, digitais, para, migrar, meu..."
43123,Tim,2020-12-30,MG,Juiz de Fora,O plano foi cancelado no dia 26/11/2020 e poss...,"""Prezado Jeferson, Em atenção à sua demanda pe...",186fab829093dcd81b85f050cddd5f79cd8d958d,"[plano, foi, cancelado, no, dia, possuia, valo..."
43124,Tim,2020-12-30,MG,Lagoa Santa,Trabalho em uma região de Lagoa Santa/MG que o...,Prezado(a) Vinicius ! Em virtude do prazo esti...,690d2f4395b261feeda5b7bded494219c1531b43,"[trabalho, em, uma, regiao, de, lagoa, santa, ..."
...,...,...,...,...,...,...,...,...
174220,Tim,2020-02-01,MG,Belo Horizonte,Recebi a conta TIM com vencimento em 10/01/202...,Weber Carneiro Gomes de Lima. Boa tarde. Em a...,7b80c7c8ea98acade5233a1865aab6349d019679,"[recebi, conta, tim, com, vencimento, em, esto..."
174232,Oi Celular,2020-02-01,MG,Além Paraíba,"BOM DIA FIZ O PLANO POS PAGO 59,90 E A PRIMEIR...","Conforme contato realizado no dia 03/01/2020, ...",740e49a6913c9d284b88ed04459d2224058aec83,"[bom, dia, fiz, plano, pos, pago, primeira, co..."
174255,Oi Fixo,2020-03-01,MG,Ribeirão das Neves,Estou procurando a empresa para renegociar min...,"Boa dia! Prezado , conforme contato realizado ...",bbe695f45625cb1da558c41d13fc6f3c70e0c897,"[estou, procurando, empresa, para, renegociar,..."
174716,Claro Celular,2020-02-01,MG,Belo Horizonte,Deixei de ser cliente Claro em novembro de 201...,"Sr.(a) ROMULO DUQUE ESTRADA CARVALHO, Em atenç...",d7dbb0babca4eadcc13aade89ec8b38fddebf981,"[deixei, de, ser, cliente, claro, em, novembro..."


### Removendo stop words

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

# conjunto de stop words a partir do modelo SpaCy
stop_words = set(nlp.Defaults.stop_words)

In [ ]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

In [ ]:
df_claims['tokens_customer_claim_clean'] = df_claims['tokens_customer_claim'].apply(remove_stopwords)
df_claims

,company_name,date,uf,city,customer_claim,company_response,id,tokens_customer_claim,tokens_customer_claim_clean
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o...",4b286594b1bfaa6c60a7f729fa0746f48c612c5d,"[boa, tarde, no, dia, por, volta, das, fui, ag...","[dia, volta, agencia, correios, localizada, ba..."
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atrav...",ba6785160fd5e43eeed575fbcdbd2abf05935cfc,"[fui, realizar, essa, ultima, atualizacao, do,...","[realizar, ultima, atualizacao, android, apare..."
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...,f607b8cecea9c11da27a6fc45f9c1bc43c1f375e,"[ao, contatar, empresa, solicitei, via, de, co...","[contatar, empresa, solicitei, via, contas, de..."
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...,04a82557dbde804b3ecf6fd357b91346c972e398,"[fiz, uma, reclamacao, anteriormente, respeito...","[fiz, reclamacao, anteriormente, respeito, deb..."
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...,a9ab2f2c4b24d64d90c2f5d00e0a85a4f5c187a4,"[solicitei, vivo, plano, para, numero, xxxxx, ...","[solicitei, vivo, plano, numero, xxxxx, aconte..."
...,...,...,...,...,...,...,...,...,...
378569,Claro Celular,2014-06-27,MG,Belo Horizonte,Em 16/05/2012 - contrataçao dos serviços de te...,"Sr. Vinicius, em atenção à reclamação sobre o ...",8cc3e46d81667f3906668ac04e8ebe0b70099c88,"[em, contratacao, dos, servicos, de, telefonia...","[contratacao, servicos, telefonia, movel, cont..."
378570,Walmart.com,2014-06-27,MG,Formiga,Na madrugada do último dia 29/05/14 efetuei um...,O Walmart.com informa que a compra do cliente ...,e35b701e491bb5fa96f3ab9be6c8561002344fd9,"[na, madrugada, do, ultimo, dia, efetuei, uma,...","[madrugada, ultimo, dia, efetuei, compra, site..."
378571,Claro Celular,2014-06-27,MG,Juiz de Fora,eu liguei para operadora pedindo para efetuar ...,"Sra. Caroline, em atenção à reclamação sobre o...",77b002e25da07990cc3d2e8876d282730ac681dc,"[eu, liguei, para, operadora, pedindo, para, e...","[liguei, operadora, pedindo, efetuar, cancelam..."
378572,Claro Celular,2014-06-27,MG,Belo Horizonte,"A CLARO, não está prestando o serviços devidos...","Sra. Rejane, em atenção à reclamação sobre cob...",bf91d708e308e5d77f801862cca7ad487b6d4b19,"[claro, nao, esta, prestando, servicos, devido...","[claro, nao, prestando, servicos, devidos, dia..."


In [ ]:
df_claims.count()

,0
company_name,378574
date,378574
uf,378574
city,378574
customer_claim,378574
company_response,375107
id,378574
tokens_customer_claim,378574
tokens_customer_claim_clean,378574


## Salvamento em parquet

In [ ]:
# Sem stop_words para análises futuras
rfzd_claims_clean = (df_claims[['tokens_customer_claim_clean', 'id']]
               .rename(columns={'tokens_customer_claim_clean': 'tokens_customer_claim'}))

rfzd_claims_clean.to_parquet('../dataset/refined/rfzd_telecom_pandemic_claims_clean.parquet.snappy', engine='pyarrow', compression='snappy')

In [9]:
rfzd_claims = (df_claims[['tokens_customer_claim', 'id']])

rfzd_claims.to_parquet('../dataset/refined/rfzd_telecom_pandemic_claims.parquet.snappy', engine='pyarrow', compression='snappy')